In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

In [2]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [3]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

这将在当前目录中生成一个 event 文件，其名称格式如下：

events.out.tfevents.{timestamp}.{hostname}

现在，在新的终端中使用以下 shell 命令启动 TensorBoard：

tensorboard --logdir .

In [4]:
sess = tf.Session()
print(sess.run(total))

7.0


In [5]:
print(sess.run({'ab':(a, b), 'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


In [6]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.8460648 0.349478  0.5894804]
[0.8007721 0.6439775 0.261886 ]
(array([1.939123 , 1.9347353, 1.8095939], dtype=float32), array([2.9391232, 2.9347353, 2.809594 ], dtype=float32))


*供给*

In [7]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [11]:
print(sess.run(z, feed_dict={x:1, y:2}))
print(sess.run(z, feed_dict={x:[1,2], y:[3,4]}))

3.0
[4. 6.]


*数据集*

In [12]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

In [13]:
while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break

[0 1]
[2 3]
[4 5]
[6 7]


*创建层*

In [14]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

*初始化层*

In [15]:
init = tf.global_variables_initializer()
sess.run(init)

In [16]:
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

[[0.10736537]
 [2.389845  ]]


*特征列*

In [17]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

In [18]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

In [19]:
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


#### 训练一个小回归

In [20]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

In [21]:
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

In [22]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[1.6182414]
 [3.2364829]
 [4.8547244]
 [6.4729657]]


In [23]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

39.3227


In [24]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [25]:
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

39.3227
27.429129
19.175564
13.447734
9.472464
6.7132626
4.7978716
3.4679868
2.5443776
1.9026797
1.456599
1.1462575
0.93010783
0.7793204
0.67389154
0.59994054
0.5478364
0.5108957
0.4844817
0.4653764
0.45134723
0.4408447
0.43279392
0.4264489
0.42129216
0.4169643
0.41321605
0.40987453
0.40681958
0.40396786
0.40126157
0.3986606
0.39613703
0.3936714
0.3912502
0.38886425
0.38650674
0.3841734
0.38186085
0.37956688
0.37729
0.37502903
0.37278318
0.37055185
0.36833462
0.3661312
0.3639413
0.36176482
0.35960144
0.3574512
0.3553138
0.3531893
0.35107765
0.34897846
0.3468919
0.34481794
0.34275624
0.34070694
0.3386699
0.336645
0.33463234
0.33263156
0.33064285
0.32866594
0.3267009
0.3247476
0.322806
0.32087606
0.31895757
0.31705055
0.3151549
0.31327072
0.31139767
0.30953592
0.3076853
0.30584565
0.30401707
0.30219936
0.3003926
0.29859656
0.29681134
0.29503673
0.29327273
0.2915193
0.28977636
0.28804383
0.28632167
0.2846098
0.28290814
0.2812167
0.2795353
0.27786407
0.27620274
0.27455136
0.27290985
0.2712

In [26]:
print(sess.run(y_pred))

[[-0.82848746]
 [-1.4014589 ]
 [-1.9744303 ]
 [-2.5474017 ]]
